In [26]:
"""
import pandas as pd
import yfinance as yf
from finvizfinance.screener.overview import Overview

def get_combined_top_stocks(countries=['USA', 'Canada'], limit_per_country=100):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching top rated stocks for: {country}...")
        
        try:
            foverview = Overview()
            
            # Use the correct filter syntax we found earlier
            filters_dict = {
                'Analyst Recom.': 'Strong Buy (1)',
                'Country': country
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- FIX STARTS HERE ---
    # 1. Inspect available columns to avoid guessing
    available_columns = combined_df.columns.tolist()
    print(f"\nDebug: Columns found in data: {available_columns}")

    # 2. Determine which 'Recommendation' column name is being used (if any)
    recom_col = None
    if 'Recom' in available_columns:
        recom_col = 'Recom'
    elif 'Analyst Recom' in available_columns:
        recom_col = 'Analyst Recom'
        
    # 3. Sort if we found the column
    if recom_col:
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # 4. safely build the list of columns to return
    # We only include columns that actually exist in 'available_columns'
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country']
    
    # Add the recommendation column to our list if it exists
    if recom_col:
        desired_cols.append(recom_col)

    # Filter columns safely
    final_cols = [c for c in desired_cols if c in available_columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
top_north_america = get_combined_top_stocks(['USA', 'Canada'])

if not top_north_america.empty:
    print(f"\nFound {len(top_north_america)} stocks.")
    print(top_north_america)
    
    # Get tickers
    ticker_list = top_north_america['Ticker'].tolist()
    print(f"\nTicker List: {ticker_list}")
else:
    print("DataFrame is empty.")
"""

    

'\nimport pandas as pd\nimport yfinance as yf\nfrom finvizfinance.screener.overview import Overview\n\ndef get_combined_top_stocks(countries=[\'USA\', \'Canada\'], limit_per_country=100):\n    all_stocks = []\n\n    for country in countries:\n        print(f"Fetching top rated stocks for: {country}...")\n\n        try:\n            foverview = Overview()\n\n            # Use the correct filter syntax we found earlier\n            filters_dict = {\n                \'Analyst Recom.\': \'Strong Buy (1)\',\n                \'Country\': country\n            }\n            foverview.set_filter(filters_dict=filters_dict)\n\n            # Get results\n            df_results = foverview.screener_view()\n\n            if not df_results.empty:\n                df_results[\'Source_Country\'] = country\n                all_stocks.append(df_results)\n            else:\n                print(f"   No stocks found for {country}.")\n\n        except Exception as e:\n            print(f"   Error fetching

In [31]:
import pandas as pd
from finvizfinance.screener.overview import Overview
#Top picks for Canada with refined filtering
def get_filtered_picks(countries=['Canada'], limit_per_country=200):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching candidates for: {country}...")
        
        try:
            foverview = Overview()
            
            # --- STEP 1: Broaden the initial search ---
            # We ask for 'Buy or better' to get a mix of Strong Buys (1.0) and regular Buys (2.0)
            filters_dict = {
                'Analyst Recom.': 'Buy or better',
                'Country': country
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- STEP 2: The "Option 2" Pandas Filter ---
    
    # A. Identify the correct column name (usually 'Recom' or 'Analyst Recom')
    cols = combined_df.columns.tolist()
    recom_col = 'Recom' if 'Recom' in cols else 'Analyst Recom'
    
    if recom_col in cols:
        # B. Convert column to numbers (force any text to become NaN)
        combined_df[recom_col] = pd.to_numeric(combined_df[recom_col], errors='coerce')
        
        # C. APPLY THE STRICT FILTER
        # Logic: 1.0 = Strong Buy, 2.0 = Buy, 3.0 = Hold.
        # We keep anything <= 2.5 to strictly exclude 'Hold' (3.0) and worse.
        print(f"\nFiltering out 'Hold' or worse (Rating > 2.0)...")
        combined_df = combined_df[combined_df[recom_col] <= 2.0]
        
        # D. Sort by best rating
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # Clean up columns for display
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country', recom_col]
    final_cols = [c for c in desired_cols if c in combined_df.columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
top_picks = get_filtered_picks(['Canada'])

if not top_picks.empty:
    print(f"\nFound {len(top_picks)} 'Buy' rated stocks.")
    
    # --- STEP 3: The View Fix ---
    # This forces VS Code/Terminal to show ALL rows
    pd.set_option('display.max_rows', None)
    
    print(top_picks)
    
    # Get tickers
    ticker_list = top_picks['Ticker'].tolist()
    #print(f"\nTicker List: {ticker_list}") # Optional: Uncomment to see list
else:
    print("DataFrame is empty.")

Fetching candidates for: Canada...
[Info] loading page [###########################---] 8/9 
Found 163 'Buy' rated stocks.
    Ticker                                Company                  Sector  \
0     ABCL                AbCellera Biologics Inc              Healthcare   
1      ACB                    Aurora Cannabis Inc              Healthcare   
2     ACHV              Achieve Life Sciences Inc              Healthcare   
3     ADUR           Aduro Clean Technologies Inc             Industrials   
4      AEM                 Agnico Eagle Mines Ltd         Basic Materials   
5       AG      First Majestic Silver Corporation         Basic Materials   
6      AGI                        Alamos Gold Inc         Basic Materials   
7      ALM                 Almonty Industries Inc         Basic Materials   
8      ASM          Avino Silver & Gold Mines Ltd         Basic Materials   
9      ATS                        ATS Corporation             Industrials   
10    AUPH            Aurinia 

c:\Users\jdcc3\AppData\Local\Programs\Python\Python313\Lib\site-packages\finvizfinance\screener\base.py:134: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame(frame)], ignore_index=True)


In [ ]:
import pandas as pd
from finvizfinance.screener.overview import Overview

def get_filtered_picks(countries=['USA'], limit_per_country=5000):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching candidates for: {country}...")
        
        try:
            foverview = Overview()
            
            # --- STEP 1: Broaden the initial search ---
            # We ask for 'Buy or better' to get a mix of Strong Buys (1.0) and regular Buys (2.0)
            filters_dict = {
                'Analyst Recom.': 'Buy or better',
                'Country': country
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- STEP 2: The "Option 2" Pandas Filter ---
    
    # A. Identify the correct column name (usually 'Recom' or 'Analyst Recom')
    cols = combined_df.columns.tolist()
    recom_col = 'Recom' if 'Recom' in cols else 'Analyst Recom'
    
    if recom_col in cols:
        # B. Convert column to numbers (force any text to become NaN)
        combined_df[recom_col] = pd.to_numeric(combined_df[recom_col], errors='coerce')
        
        # C. APPLY THE STRICT FILTER
        # Logic: 1.0 = Strong Buy, 2.0 = Buy, 3.0 = Hold.
        # We keep anything <= 2.5 to strictly exclude 'Hold' (3.0) and worse.
        print(f"\nFiltering out 'Hold' or worse (Rating > 2.0)...")
        combined_df = combined_df[combined_df[recom_col] < 1.5]
        
        # D. Sort by best rating
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # Clean up columns for display
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country', recom_col]
    final_cols = [c for c in desired_cols if c in combined_df.columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
top_picks = get_filtered_picks(['USA'])

if not top_picks.empty:
    print(f"\nFound {len(top_picks)} 'Buy' rated stocks.")
    
    # --- STEP 3: The View Fix ---
    # This forces VS Code/Terminal to show ALL rows
    #pd.set_option('display.max_rows', None)
    
    print(top_picks)
    
    # Get tickers
    ticker_list = top_picks['Ticker'].tolist()
    #print(f"\nTicker List: {ticker_list}") # Optional: Uncomment to see list
else:
    print("DataFrame is empty.")

Fetching candidates for: USA...
[Info] loading page [##############################] 149/150 
Found 2000 'Buy' rated stocks.
      Ticker                                        Company  \
0          A                       Agilent Technologies Inc   
1         AA                                     Alcoa Corp   
2        AAL                    American Airlines Group Inc   
3       AAOI                    Applied Optoelectronics Inc   
4       AAON                                       AAON Inc   
5       AAPL                                      Apple Inc   
6       AARD                      Aardvark Therapeutics Inc   
7       ABAT            American Battery Technology Company   
8       ABBV                                     Abbvie Inc   
9       ABCB                                 Ameris Bancorp   
10      ABEO                        Abeona Therapeutics Inc   
11       ABL                   Abacus Global Management Inc   
12       ABM                             ABM Industries 

In [ ]:
import pandas as pd
from finvizfinance.screener.overview import Overview

def get_filtered_picks(countries=['USA'], limit_per_country=5000):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching candidates for: {country}...")
        
        try:
            foverview = Overview()
            
            # --- STEP 1: Broaden the initial search ---
            filters_dict = {
                'Analyst Recom.': 'Buy or better',
                'Country': country
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- STEP 2: The "Option 2" Pandas Filter ---
    cols = combined_df.columns.tolist()
    recom_col = 'Recom' if 'Recom' in cols else 'Analyst Recom'
    
    if recom_col in cols:
        # Convert column to numbers
        combined_df[recom_col] = pd.to_numeric(combined_df[recom_col], errors='coerce')
        
        # Filter: keep anything <= 1.5 (Strong Buys)
        print(f"Filtering out 'Hold' or worse (Rating > 1.5)...")
        combined_df = combined_df[combined_df[recom_col] < 1.5]
        
        # Sort by best rating
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # Clean up columns
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country', recom_col]
    final_cols = [c for c in desired_cols if c in combined_df.columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
top_picks = get_filtered_picks(['USA'])

if not top_picks.empty:
    # Ensure Price is numeric (Crucial for sorting in Data Viewer)
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    print(f"\nSuccess! Found {len(top_picks)} candidates.")
    print("Variable 'top_picks' is ready for the Data Viewer.")

else:
    print("DataFrame is empty.")

Fetching candidates for: USA...


In [ ]:

if not top_picks.empty:
    # Ensure the 'Price' column is numeric (removes any potential formatting errors)
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    # Filter for stocks with Price < 30
    cheap_picks = top_picks[top_picks['Price'] < 30]

    print(f"\nFound {len(cheap_picks)} stocks under $30.")
    print(cheap_picks)
else:
    print("DataFrame is empty.")

In [28]:
import yfinance as yf
import time

# Loop through EVERY symbol in the list (removed the [:3])
for symbol in ticker_list:
    
    # 1. Fetch data
    stock = yf.Ticker(symbol)
    print(f"\n--- Checking {symbol} ---")
    
    try:
        # 2. Get the upgrades/downgrades
        upgrades = stock.upgrades_downgrades
        
        if not upgrades.empty:
            # Filter for 2025 actions using the .index fix
            recent = upgrades[upgrades.index >= '2025-11-01']
            
            if not recent.empty:
                print(recent)
            else:
                print(f"No actions for {symbol} in 2025.")
        else:
            print(f"No analyst history found for {symbol}.")
            
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")

    # 3. CRITICAL: Sleep for 1 second to avoid getting banned
    time.sleep(1)


--- Checking A ---
                                  Firm       ToGrade     FromGrade Action  \
GradeDate                                                                   
2025-11-25 19:04:11          Citigroup           Buy           Buy   main   
2025-11-25 17:52:33                UBS           Buy           Buy   main   
2025-11-25 15:39:43        Wells Fargo    Overweight    Overweight   main   
2025-11-25 15:10:42          JP Morgan    Overweight    Overweight   main   
2025-11-25 14:42:29           Barclays  Equal-Weight  Equal-Weight   main   
2025-11-25 13:14:47              Baird    Outperform    Outperform   main   
2025-11-25 12:05:56  B of A Securities       Neutral       Neutral   main   

                    priceTargetAction  currentPriceTarget  priorPriceTarget  
GradeDate                                                                    
2025-11-25 19:04:11            Raises               185.0             165.0  
2025-11-25 17:52:33            Raises               

HTTP Error 404: 



--- Checking ACFN ---
No analyst history found for ACFN.

--- Checking ACHC ---
                                  Firm       ToGrade     FromGrade Action  \
GradeDate                                                                   
2025-11-18 15:00:16            Keybanc    Overweight    Overweight   main   
2025-11-12 14:19:42        RBC Capital    Outperform    Outperform   main   
2025-11-10 14:53:42         Guggenheim           Buy           Buy   main   
2025-11-07 18:01:18                UBS           Buy           Buy   main   
2025-11-07 16:07:33  Cantor Fitzgerald       Neutral       Neutral   main   
2025-11-06 14:21:41           Barclays  Equal-Weight  Equal-Weight   main   

                    priceTargetAction  currentPriceTarget  priorPriceTarget  
GradeDate                                                                    
2025-11-18 15:00:16            Lowers                30.0              35.0  
2025-11-12 14:19:42            Lowers                22.0           

KeyboardInterrupt: 